In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df1 = pd.read_excel(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\CRIS_UnLoadingData\CRIS_UnLoadingData\XXHIL_CRIS_UNLOADING_DTL_Nov2019_2020.xlsx")

In [4]:
df2 = pd.read_excel(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\CRIS_UnLoadingData\CRIS_UnLoadingData\XXHIL_CRIS_UNLOADING_DTL_2021.xlsx")

C:\Users\nikhil.upadhyay-v\AppData\Local\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
df3 = pd.read_excel(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\CRIS_UnLoadingData\CRIS_UnLoadingData\XXHIL_CRIS_UNLOADING_DTL_2022.xlsx")

In [6]:
df4 = pd.read_excel(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\CRIS_UnLoadingData\CRIS_UnLoadingData\XXHIL_CRIS_UNLOADING_DTL_01Jan2023_31Aug2023.xlsx")

C:\Users\nikhil.upadhyay-v\AppData\Local\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
combined_df = pd.concat([df1,df2,df3,df4])

In [8]:
combined_df.shape

(162748, 28)

In [9]:
combined_df = combined_df.drop_duplicates()

In [10]:
combined_df.shape

(162748, 28)

In [11]:
combined_df['NormalizedWagonType'] = np.where(combined_df['WAGONTYPE'].str.contains(r'^BCN[0-9A-Z]*$'), 'BCN', combined_df['WAGONTYPE'].str[:4])

In [12]:
combined_df.shape

(162748, 29)

In [13]:
combined_df.to_csv('combined_df.csv', index= False)

In [14]:
combined_unloading_group = combined_df.groupby(['FNR', 'COMMODITYCODE', 'CONSIGNEENAME', 'DIVISION', 'STATION', 'STATIONFROM', 'STATIONTO', 'NormalizedWagonType']).agg({
                    'INVOICENUMBER': ['first'], 'INVOICEDATE': ['max', 'count'],'ACTUALWEIGHT': ['sum','count'], 'COMMERCIALPLACEMENTTIME': 'max', 'REMOVALTIME': 'max', 'ATTRIBUTE4': 'first'
}).reset_index()

In [15]:
combined_unloading_group.columns = ['FNR', 'COMMODITYCODE', 'CONSIGNEENAME', 'DIVISION', 'STATION', 'STATIONFROM', 'STATIONTO', 'NormalizedWagonType',
                                   'INVOICENUMBER', 'INVOICEDATE','NumOfWagons', 'ACTUALWEIGHT_sum', 'ACTUALWEIGHT_count', 'COMMERCIALPLACEMENTTIME',
                                   'REMOVALTIME', 'ATTRIBUTE4']

In [16]:
combined_unloading_group = combined_unloading_group[~(combined_unloading_group['FNR'] == 18031010408)]

In [17]:
combined_unloading_group.shape

(2835, 16)

In [18]:
combined_unloading_group.to_csv('combined_unloading_group.csv', index= False)

Indent data

In [94]:
indent_df = pd.read_csv(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\XXHIL_CRIS_INDENT_DTL_1stJan2021_21stAug2023.csv")
indent_df.head()
indent_df.drop('DEMANDDTLSID', axis= 1, inplace= True)

In [95]:
indent_group = indent_df.groupby('FNRNUMBER').agg({
    'NOOFUNITS': 'sum',
    'QUANTITY': 'max'
}).reset_index()

In [96]:
#merging the unloading and indent data
merged_unloading_indent = pd.merge(combined_unloading_group, indent_group, left_on= 'FNR', right_on= 'FNRNUMBER', how= 'left')

In [97]:
merged_unloading_indent.to_csv('merged_unloading_indent.csv', index= False)

In [98]:
numeric_code_df = pd.read_csv(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\Numeric Code with Commodity Name.csv")

In [99]:
#drop duplicates
numeric_code_df = numeric_code_df.drop_duplicates(['SUB COMMODITY DETAIL CODE'])
numeric_code_df.shape

(540, 2)

In [100]:
#merging the unloading_indent data with numeric codes
merged_ui_commodity = pd.merge(merged_unloading_indent, numeric_code_df, left_on = ['COMMODITYCODE'], right_on= ['SUB COMMODITY DETAIL CODE'], how= 'left')

In [101]:
merged_ui_commodity['SUB COMMODITY DETAIL NAME'].unique()

array(['RUN OF MINES (ROM) COAL (F)', 'RUN OF MINES (ROM) COAL (D)',
       'RUN OFF MINES COAL(ROM)', 'RUN OF MINES (ROM) COAL (A)',
       'CAUSTIC SODA LYE', 'BAUXITE', 'WASHED COAL',
       'CALCINED PETROLEUM COKE', nan, 'RUN OF MINES (ROM) COAL (C)',
       'ALUMINA', 'STEAM COAL (IMPORTED)', 'RUN OF MINES (ROM) COAL (E)',
       'RED MUD', 'WASHERY REJECT', 'BURNT LIME', 'QUICK LIME', 'FLY ASH',
       'RUN OF MINES (ROM) COAL (B)', 'PATENT FUEL (AS COAL)',
       'STEAM COAL', 'ASHES'], dtype=object)

In [102]:
#mapping the values:
def map_commodity(commodity):
    if 'COAL' in str(commodity).upper():
        return 'COAL'
    elif 'COKE' in str(commodity).upper():
        return 'COKE'
    elif 'LIME' in str(commodity).upper():
        return 'LIME'
    elif 'RED MUD' in str(commodity).upper():
        return 'RED MUD'
    elif 'CAUSTIC' in str(commodity).upper():
        return 'CAUSTIC'
    elif 'FLY ASH' in str(commodity).upper():
        return 'FLY ASH'
    elif 'BAUXITE' in str(commodity).upper():
        return 'BAUXITE'
    elif 'FO' in str(commodity).upper():
        return 'FO'
    elif 'F.O.' in str(commodity).upper():
        return 'FO'
    else:
        return 'Any commodity'
    
merged_ui_commodity['CommodityName'] = merged_ui_commodity['SUB COMMODITY DETAIL NAME'].apply(map_commodity)

In [103]:
merged_ui_commodity.to_csv('merged_ui_commodity.csv', index= False)

In [104]:
def map_commodities(row):
    if (row['CONSIGNEENAME'] == 'HIL'): 
        if (row['STATION'] == 'HIMB'):
            if('FLY ASH' in row['SUB COMMODITY DETAIL NAME modified']):
                return 'FLY ASH'
            elif 'COKE' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'COKE'
            else:
                return 'Any commodity'

        elif (row['STATION'] == 'IACM'):
            if 'CAUSTIC' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'CAUSTIC'
            elif 'RED MUD' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'RED MUD'
            elif 'F.O.' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'FO'
            elif 'FLY ASH' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'FLY ASH'
            else:
                return 'Any commodity'

        elif (row['STATION'] == 'HACG'):
            if 'COAL' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'COAL'    
            elif 'BAUXITE' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'BAUXITE'  
            elif 'FO' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'FO'
            elif 'CAUSTIC' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'CAUSTIC'
            elif 'FLY ASH' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'FLY ASH'
            elif 'RED MUD' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'RED MUD'
            elif 'COKE' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'COKE'
            else:
                return 'Any commodity'

        elif (row['STATION'] == 'HIWS'):
            if 'COAL' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'COAL'        
            elif 'FLY ASH' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'FLY ASH'
            else:
                return 'Any commodity'

    elif (row['CONSIGNEENAME'] == 'HILD'):
        if (row['STATION'] == 'MAAL'):
            if 'COKE' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'COKE'
            elif 'FLY ASH' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'FLY ASH'
            else:
                return 'Any commodity'

    elif (row['CONSIGNEENAME'] == 'UAIL'):
        if (row['STATION'] == 'MUAT'):
            if 'LIME' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'LIME'
            elif 'FLY ASH' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'FLY ASH'
            elif 'FO' in row['SUB COMMODITY DETAIL NAME modified']:
                return 'FO'
            else:
                return 'Any commodity'



In [105]:
merged_ui_commodity['SUB COMMODITY DETAIL NAME'].fillna("NULL", inplace= True)

In [106]:
merged_ui_commodity['SUB COMMODITY DETAIL NAME modified'] = merged_ui_commodity['SUB COMMODITY DETAIL NAME'].astype(str)         
merged_ui_commodity['CommodityName modified'] = merged_ui_commodity.apply(map_commodities, axis=1)

In [107]:
#replacing Null values in the CommodityName to Any commodity: this will be fixed once master table is updated.
merged_ui_commodity['CommodityName modified'].fillna('Any commodity', inplace= True)

In [108]:
merged_ui_commodity.shape

(2835, 24)

In [109]:
merged_ui_commodity.to_csv('merged_ui_commodity_modified.csv', index= False)

In [110]:
#loading the Freetime dataset
freetime_df = pd.read_csv(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\Free_time_data_with_wagontype.csv")
freetime_df

,Unit Name,Consigner / Consignee Code,Siding/Station Code,Type of Wagon,Commodity,Unloading / Loading,Free Hours
0,Utkal,UAIL,MUAT,BOXN,Any commodity,Unloading,5
1,Utkal,UAIL,MUAT,BCN,LIME,Unloading,6
2,Utkal,UAIL,MUAT,BTPN,FO,Unloading,5
3,Utkal,UAIL,MUAT,BOXN,FLY ASH,Loading,4
4,Mahan,HIL,HIMB,BOXN,Any commodity,Unloading,5
5,Mahan,HIL,HIMB,BCN,CP COKE,Unloading,6
6,Mahan,HIL,HIMB,BOXN,FLY ASH,Loading,4
7,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5
8,Aditya,HILD,MAAL,BCN,CP COKE,Unloading,6
9,Aditya,HILD,MAAL,BOXN,FLY ASH,Loading,4


In [111]:
#splitting the commodity column
split_rows = []
for index, row in freetime_df.iterrows():
    commodities = row['Commodity'].split('/')
    
    # For each commodity in the split, create a new row with the same values
    for commodity in commodities:
        new_row = row.copy()
        new_row['Commodity'] = commodity.strip()  # Remove leading/trailing spaces
        split_rows.append(new_row)

# Create a new DataFrame with the split rows
freetime_df = pd.DataFrame(split_rows)

freetime_df.reset_index(drop=True, inplace=True)
freetime_df

,Unit Name,Consigner / Consignee Code,Siding/Station Code,Type of Wagon,Commodity,Unloading / Loading,Free Hours
0,Utkal,UAIL,MUAT,BOXN,Any commodity,Unloading,5
1,Utkal,UAIL,MUAT,BCN,LIME,Unloading,6
2,Utkal,UAIL,MUAT,BTPN,FO,Unloading,5
3,Utkal,UAIL,MUAT,BOXN,FLY ASH,Loading,4
4,Mahan,HIL,HIMB,BOXN,Any commodity,Unloading,5
5,Mahan,HIL,HIMB,BCN,CP COKE,Unloading,6
6,Mahan,HIL,HIMB,BOXN,FLY ASH,Loading,4
7,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5
8,Aditya,HILD,MAAL,BCN,CP COKE,Unloading,6
9,Aditya,HILD,MAAL,BOXN,FLY ASH,Loading,4


In [112]:
#merging the uic and the freetime
merged_uic_freetime = pd.merge(merged_ui_commodity, freetime_df, left_on= ['CONSIGNEENAME', 'STATION', 'NormalizedWagonType', 'CommodityName modified'], right_on= 
         ['Consigner / Consignee Code', 'Siding/Station Code','Type of Wagon', 'Commodity'], how= 'left' )
merged_uic_freetime

,FNR,COMMODITYCODE,CONSIGNEENAME,DIVISION,STATION,STATIONFROM,STATIONTO,NormalizedWagonType,INVOICENUMBER,INVOICEDATE,NumOfWagons,ACTUALWEIGHT_sum,ACTUALWEIGHT_count,COMMERCIALPLACEMENTTIME,REMOVALTIME,ATTRIBUTE4,FNRNUMBER,NOOFUNITS,QUANTITY,SUB COMMODITY DETAIL NAME,SUB COMMODITY DETAIL CODE,CommodityName,SUB COMMODITY DETAIL NAME modified,CommodityName modified,Unit Name,Consigner / Consignee Code,Siding/Station Code,Type of Wagon,Commodity,Unloading / Loading,Free Hours
0,18031612075,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,10,03-05-2020,59,4097.45,59,04-05-20 17:30,04-05-20 22:30,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0
1,18032707097,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,14,05-05-2020,58,4053.00,58,06-05-20 02:35,06-05-20 10:35,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0
2,18042409592,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,16,06-05-2020,59,3946.26,59,07-05-20 04:20,07-05-20 09:20,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0
3,18061704498,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,21,09-05-2020,58,3972.20,58,10-05-20 00:00,11-05-20 05:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0
4,18062105597,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,38,16-03-2020,59,4197.05,59,17-03-20 12:20,17-03-20 18:20,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830,23082804032,4011208,DALM,CKP,OCIG,MAAL,OCIG,BOXN,2,28-08-2023,51,3396.98,51,29-08-23 12:30,29-08-23 20:30,Manual,2.308280e+10,59.0,4070.0,FLY ASH,4011208.0,FLY ASH,FLY ASH,Any commodity,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2831,23082813580,2991323,HIL,JBP,HIMB,RSCR,HIMB,BOXN,26,29-08-2023,58,3774.24,58,31-08-23 08:45,31-08-23 13:45,Manual,2.308281e+10,59.0,4000.0,RUN OFF MINES COAL(ROM),2991323.0,COAL,RUN OFF MINES COAL(ROM),Any commodity,Mahan,HIL,HIMB,BOXN,Any commodity,Unloading,5.0
2832,23082818992,2991323,HIL,DHN,HACG,BLCB,HACG,BOXN,22,29-08-2023,59,4090.34,59,30-08-23 11:35,30-08-23 21:30,Manual,2.308282e+10,61.0,3894.0,RUN OFF MINES COAL(ROM),2991323.0,COAL,RUN OFF MINES COAL(ROM),COAL,Renukoot,HIL,HACG,BOXN,COAL,Unloading,7.0
2833,23082904040,1051110,HIL,DHN,HACG,MMRC,HACG,BOXN,30,30-08-2023,56,3735.95,56,30-08-23 22:25,31-08-23 03:55,Manual,2.308290e+10,59.0,4012.0,BAUXITE,1051110.0,BAUXITE,BAUXITE,BAUXITE,Renukoot,HIL,HACG,BOXN,BAUXITE,Unloading,7.0


In [113]:
from datetime import datetime
date_format = '%d-%m-%y %H:%M'

merged_uic_freetime['REMOVALTIME'] = merged_uic_freetime['REMOVALTIME'].apply(lambda x: datetime.strptime(x, date_format))
merged_uic_freetime['COMMERCIALPLACEMENTTIME'] = merged_uic_freetime['COMMERCIALPLACEMENTTIME'].apply(lambda x: datetime.strptime(x, date_format))



In [114]:
# merged_uic_freetime['REMOVALTIME'] = pd.to_datetime(merged_uic_freetime['REMOVALTIME'], format= '%d-%m-%Y %H:%M')
# merged_uic_freetime['COMMERCIALPLACEMENTTIME'] =pd.to_datetime(merged_uic_freetime['COMMERCIALPLACEMENTTIME'], format= '%d-%m-%Y %H:%M')

merged_uic_freetime['TAT'] = merged_uic_freetime['REMOVALTIME'] - merged_uic_freetime['COMMERCIALPLACEMENTTIME']
merged_uic_freetime['TAT'] = np.ceil(merged_uic_freetime['TAT'].dt.total_seconds()/3600)
merged_uic_freetime['TAT']

0        5.0
1        8.0
2        5.0
3       29.0
4        6.0
        ... 
2830     8.0
2831     5.0
2832    10.0
2833     6.0
2834     7.0
Name: TAT, Length: 2835, dtype: float64

In [115]:
merged_uic_freetime.to_csv('merged_uic_freetime.csv', index= False)

In [116]:
merged_uic_freetime['Demurrage_Time'] = merged_uic_freetime['TAT'] - merged_uic_freetime['Free Hours']

In [117]:
merged_uic_freetime['Demurrage_Cost'] = merged_uic_freetime['Demurrage_Time']*150*merged_uic_freetime['NOOFUNITS']*1.05

In [118]:
merged_uic_freetime['Demurrage_Cost'] = np.where(
    merged_uic_freetime['Demurrage_Cost']<0, 0, merged_uic_freetime['Demurrage_Cost'])

In [119]:
bin_ranges = [0, 20, 40, 60, 80, 100, 1000]
bin_labels = ["0-20 hrs", "21-40 hrs", "41-60 hrs", "61-80 hrs", "81-100 hrs", "100+ hrs"]

merged_uic_freetime['TAT_Categories'] = pd.cut(merged_uic_freetime['TAT'], bins= bin_ranges, labels=bin_labels, right=False)

In [120]:
demurrage_data = pd.read_excel(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\XXHIL_CRIS_DEMURRAGE_DTL_1stJan2023-31stAug2023.xlsx")
demurrage_data

C:\Users\nikhil.upadhyay-v\AppData\Local\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,FNR,CGST,ECESS,HECESS,IGST,KKCESS,SBCESS,SGST,UTGST,APPLIEDSTATION,CONSIGNEE,CONSIGNOR,DEMURRAGECHARGECOLLECTED,DEMURRAGECHARGEDUE,DEMURRAGECHARGEOUTSTATNDING,DEMURRAGECHARGEWAIVED,DIVISION,DMRGHRS,FNOTEDATE,FNOTENUMBER,FREETIME,INVOICEDATE,INVOICENUMBER,SERVICETAX,STATIONFROM,STATIONTO,TOTALGST,TOTALSERVICETAX,CREATIONDATE,ATTRIBUTE1,ATTRIBUTE2,ATTRIBUTE3,ATTRIBUTE4,ATTRIBUTE5
0,21022618761,0.00,0,0,0.0,0,0,0.00,0,MUAT,UAIL,MCFL,0,8700,0,8700,WAT,0.0,26-02-2021,239.001,7:00,05-03-2021,6,0,LOMB,MUAT,0,0,2023-09-07 12:43:26,141218990,06-03-2021,10-03-2021,Manual,NaN
1,21022815920,0.00,0,0,0.0,0,0,0.00,0,MUAT,UAIL,MCFL,0,150450,0,150450,WAT,0.0,28-02-2021,1701.001,5:00,07-03-2021,12,0,ACTR,MUAT,0,0,2023-09-07 12:43:26,141218990,06-03-2021,10-03-2021,Manual,NaN
2,21022815920,0.00,0,0,0.0,0,0,0.00,0,MUAT,UAIL,MCFL,0,150450,0,150450,WAT,0.0,28-02-2021,1701.001,5:00,07-03-2021,12,0,ACTR,MUAT,0,0,2023-09-07 12:43:26,141218990,06-03-2021,10-03-2021,Manual,NaN
3,21022618761,0.00,0,0,0.0,0,0,0.00,0,MUAT,UAIL,MCFL,0,8700,0,8700,WAT,0.0,26-02-2021,239.001,7:00,05-03-2021,6,0,LOMB,MUAT,0,0,2023-09-07 12:43:26,141218990,06-03-2021,10-03-2021,Manual,NaN
4,21022214787,0.00,0,0,0.0,0,0,0.00,0,MUAT,UAIL,UAIL,0,123900,0,123900,WAT,12.0,22-02-2021,27.001,7:00,07-03-2021,11,0,BBSB,MUAT,0,0,2023-09-07 12:43:26,141218990,06-03-2021,10-03-2021,Manual,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84011,23090402936,0.00,0,0,0.0,0,0,0.00,0,UCLM,UTCL,HIL,0,43500,43500,0,JBP,5.0,04-09-2023,117.001,7:00,04-09-2023,10,0,HACG,UCLM,0,0,2023-09-08 08:58:38,142418258,07-09-2023,07-09-2023,Scheduled,NaN
84012,22112702234,221.25,0,0,0.0,0,0,221.25,0,MAAL,HILD,MCFL,8850,8850,0,0,SBP,1.0,27-11-2022,677.001,5:00,05-09-2023,9,0,ACTR,MAAL,443,0,2023-09-08 08:58:38,142418258,07-09-2023,07-09-2023,Scheduled,NaN
84013,22082909091,0.00,0,0,0.0,0,0,0.00,0,KMKA,HILD,SECL,0,17400,17400,0,BSP,1.0,29-08-2022,2056.001,3:00,07-09-2023,22,0,KMKA,MAAL,0,0,2023-09-08 08:58:38,142418258,07-09-2023,07-09-2023,Scheduled,NaN
84014,22112702234,221.25,0,0,0.0,0,0,221.25,0,MAAL,HILD,MCFL,8850,8850,0,0,SBP,1.0,27-11-2022,677.001,5:00,05-09-2023,9,0,ACTR,MAAL,443,0,2023-09-08 08:58:38,142418258,07-09-2023,07-09-2023,Scheduled,NaN


In [121]:
demurrage_data_group = demurrage_data.groupby(['FNR', 'CONSIGNOR'])['DEMURRAGECHARGEDUE'].max().reset_index()
demurrage_data_group

,FNR,CONSIGNOR,DEMURRAGECHARGEDUE
0,20062708636,SECL,17400
1,20092805158,SECL,17700
2,20102305326,SECL,17700
3,20102305335,SECL,42750
4,20112705464,SECL,8550
...,...,...,...
856,23090515610,HIL,8850
857,23090516479,HIL,43500
858,23090604710,HIL,8700
859,23090608567,HIL,52200


In [122]:
merged_final_with_demurrage = pd.merge(merged_uic_freetime, demurrage_data_group, on= 'FNR', how= 'left' )

In [156]:
merged_final_with_demurrage

,FNR,COMMODITYCODE,CONSIGNEENAME,DIVISION,STATION,STATIONFROM,STATIONTO,NormalizedWagonType,INVOICENUMBER,INVOICEDATE,NumOfWagons,ACTUALWEIGHT_sum,ACTUALWEIGHT_count,COMMERCIALPLACEMENTTIME,REMOVALTIME,ATTRIBUTE4,FNRNUMBER,NOOFUNITS,QUANTITY,SUB COMMODITY DETAIL NAME,SUB COMMODITY DETAIL CODE,CommodityName,SUB COMMODITY DETAIL NAME modified,CommodityName modified,Unit Name,Consigner / Consignee Code,Siding/Station Code,Type of Wagon,Commodity,Unloading / Loading,Free Hours,TAT,Demurrage_Time,Demurrage_Cost,TAT_Categories,CONSIGNOR,DEMURRAGECHARGEDUE
0,18031612075,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,10,03-05-2020,59,4097.45,59,2020-05-04 17:30:00,2020-05-04 22:30:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0,5.0,0.0,NaN,0-20 hrs,NaN,NaN
1,18032707097,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,14,05-05-2020,58,4053.00,58,2020-05-06 02:35:00,2020-05-06 10:35:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0,8.0,3.0,NaN,0-20 hrs,NaN,NaN
2,18042409592,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,16,06-05-2020,59,3946.26,59,2020-05-07 04:20:00,2020-05-07 09:20:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0,5.0,0.0,NaN,0-20 hrs,NaN,NaN
3,18061704498,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,21,09-05-2020,58,3972.20,58,2020-05-10 00:00:00,2020-05-11 05:00:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0,29.0,24.0,NaN,21-40 hrs,NaN,NaN
4,18062105597,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,38,16-03-2020,59,4197.05,59,2020-03-17 12:20:00,2020-03-17 18:20:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0,6.0,1.0,NaN,0-20 hrs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830,23082804032,4011208,DALM,CKP,OCIG,MAAL,OCIG,BOXN,2,28-08-2023,51,3396.98,51,2023-08-29 12:30:00,2023-08-29 20:30:00,Manual,2.308280e+10,59.0,4070.0,FLY ASH,4011208.0,FLY ASH,FLY ASH,Any commodity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,0-20 hrs,HILD,22950.0
2831,23082813580,2991323,HIL,JBP,HIMB,RSCR,HIMB,BOXN,26,29-08-2023,58,3774.24,58,2023-08-31 08:45:00,2023-08-31 13:45:00,Manual,2.308281e+10,59.0,4000.0,RUN OFF MINES COAL(ROM),2991323.0,COAL,RUN OFF MINES COAL(ROM),Any commodity,Mahan,HIL,HIMB,BOXN,Any commodity,Unloading,5.0,5.0,0.0,0.0,0-20 hrs,NaN,NaN
2832,23082818992,2991323,HIL,DHN,HACG,BLCB,HACG,BOXN,22,29-08-2023,59,4090.34,59,2023-08-30 11:35:00,2023-08-30 21:30:00,Manual,2.308282e+10,61.0,3894.0,RUN OFF MINES COAL(ROM),2991323.0,COAL,RUN OFF MINES COAL(ROM),COAL,Renukoot,HIL,HACG,BOXN,COAL,Unloading,7.0,10.0,3.0,28822.5,0-20 hrs,CCL,26550.0
2833,23082904040,1051110,HIL,DHN,HACG,MMRC,HACG,BOXN,30,30-08-2023,56,3735.95,56,2023-08-30 22:25:00,2023-08-31 03:55:00,Manual,2.308290e+10,59.0,4012.0,BAUXITE,1051110.0,BAUXITE,BAUXITE,BAUXITE,Renukoot,HIL,HACG,BOXN,BAUXITE,Unloading,7.0,6.0,-1.0,0.0,0-20 hrs,NaN,NaN


In [123]:
siding_code_df = pd.read_excel(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\MASTER FILES - CRIS DATA.xlsx", sheet_name= 'SIDING CODE')

In [153]:
merged_final_with_demurrage_siding = pd.merge(merged_final_with_demurrage, siding_code_df, on= ['STATION'], how = 'left')

In [125]:
consignee_df = pd.read_excel(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\MASTER FILES - CRIS DATA.xlsx", sheet_name= 'CONSIGNEE NAME')

In [154]:
merged_final_with_demurrage_siding_consignee = pd.merge(merged_final_with_demurrage_siding, consignee_df, on= ['CONSIGNEENAME', 'STATION'], how= 'left')

In [127]:
consignor_df = pd.read_excel(r"C:\Users\nikhil.upadhyay-v\Desktop\CRIS POC\Data\Consignor_details.xlsx")

In [128]:
consignor_df

,CONSIGNOR,STATION,CONSIGNOR NAME
0,ACC,ACCG,THIRD PARTY - TIKARIA CEMENT WORKS SDG
1,ACC,LKES,THIRD PARTY - ASSOCIATED CEMENT CO. LTD
2,ACC,CESG,THIRD PARTY - ASSOCIATED CEMENT CO. LTD
3,ACC,KART,THIRD PARTY - KIRATPUR SAHIB
4,ACCB,CFDS,THIRD PARTY - ASSOCIATED CEMENT CO. LTD
...,...,...,...
90,HIL,MSH,MAHESANA
91,HIL,BEHR,BEOHARI
92,HIL,BE,BAREILLY
93,WCSG,SMP,THIRD PARTY - WONDER CEMENT SIDING


In [155]:
merged_final_with_demurrage_siding_consignee

,FNR,COMMODITYCODE,CONSIGNEENAME,DIVISION,STATION,STATIONFROM,STATIONTO,NormalizedWagonType,INVOICENUMBER,INVOICEDATE,NumOfWagons,ACTUALWEIGHT_sum,ACTUALWEIGHT_count,COMMERCIALPLACEMENTTIME,REMOVALTIME,ATTRIBUTE4,FNRNUMBER,NOOFUNITS,QUANTITY,SUB COMMODITY DETAIL NAME,SUB COMMODITY DETAIL CODE,CommodityName,SUB COMMODITY DETAIL NAME modified,CommodityName modified,Unit Name,Consigner / Consignee Code,Siding/Station Code,Type of Wagon,Commodity,Unloading / Loading,Free Hours,TAT,Demurrage_Time,Demurrage_Cost,TAT_Categories,CONSIGNOR,DEMURRAGECHARGEDUE,SIDING NAME,CONSIGNEE NAME
0,18031612075,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,10,03-05-2020,59,4097.45,59,2020-05-04 17:30:00,2020-05-04 22:30:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0,5.0,0.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM
1,18032707097,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,14,05-05-2020,58,4053.00,58,2020-05-06 02:35:00,2020-05-06 10:35:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0,8.0,3.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM
2,18042409592,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,16,06-05-2020,59,3946.26,59,2020-05-07 04:20:00,2020-05-07 09:20:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0,5.0,0.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM
3,18061704498,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,21,09-05-2020,58,3972.20,58,2020-05-10 00:00:00,2020-05-11 05:00:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0,29.0,24.0,NaN,21-40 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM
4,18062105597,2991130,HILD,SBP,MAAL,NKCR,MAAL,BOXN,38,16-03-2020,59,4197.05,59,2020-03-17 12:20:00,2020-03-17 18:20:00,Manual,NaN,NaN,NaN,RUN OF MINES (ROM) COAL (F),2991130.0,COAL,RUN OF MINES (ROM) COAL (F),Any commodity,Aditya,HILD,MAAL,BOXN,Any commodity,Unloading,5.0,6.0,1.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830,23082804032,4011208,DALM,CKP,OCIG,MAAL,OCIG,BOXN,2,28-08-2023,51,3396.98,51,2023-08-29 12:30:00,2023-08-29 20:30:00,Manual,2.308280e+10,59.0,4070.0,FLY ASH,4011208.0,FLY ASH,FLY ASH,Any commodity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,0-20 hrs,HILD,22950.0,"ORISSA CEMENT LTD, RAJGANGPUR (JSG-ROU)","THIRD PARTY - ORISSA CEMENT LTD, RAJGANGPUR"
2831,23082813580,2991323,HIL,JBP,HIMB,RSCR,HIMB,BOXN,26,29-08-2023,58,3774.24,58,2023-08-31 08:45:00,2023-08-31 13:45:00,Manual,2.308281e+10,59.0,4000.0,RUN OFF MINES COAL(ROM),2991323.0,COAL,RUN OFF MINES COAL(ROM),Any commodity,Mahan,HIL,HIMB,BOXN,Any commodity,Unloading,5.0,5.0,0.0,0.0,0-20 hrs,NaN,NaN,HINDALCO LTD - MAHAN,MAHAN ALUMINIUM
2832,23082818992,2991323,HIL,DHN,HACG,BLCB,HACG,BOXN,22,29-08-2023,59,4090.34,59,2023-08-30 11:35:00,2023-08-30 21:30:00,Manual,2.308282e+10,61.0,3894.0,RUN OFF MINES COAL(ROM),2991323.0,COAL,RUN OFF MINES COAL(ROM),COAL,Renukoot,HIL,HACG,BOXN,COAL,Unloading,7.0,10.0,3.0,28822.5,0-20 hrs,CCL,26550.0,HINDALCO LTD -RENUKOOT,RENUKOOT
2833,23082904040,1051110,HIL,DHN,HACG,MMRC,HACG,BOXN,30,30-08-2023,56,3735.95,56,2023-08-30 22:25:00,2023-08-31 03:55:00,Manual,2.308290e+10,59.0,4012.0,BAUXITE,1051110.0,BAUXITE,BAUXITE,BAUXITE,Renukoot,HIL,HACG,BOXN,BAUXITE,Unloading,7.0,6.0,-1.0,0.0,0-20 hrs,NaN,NaN,HINDALCO LTD -RENUKOOT,RENUKOOT


In [129]:
merged_final_with_demurrage_siding_consignee = pd.merge(merged_final_with_demurrage_siding_consignee, consignor_df,
                                                        on= ['CONSIGNOR', 'STATION'], how = 'left')

In [130]:
merged_final_with_demurrage_siding_consignee.columns

Index(['FNR', 'COMMODITYCODE', 'CONSIGNEENAME', 'DIVISION', 'STATION',
       'STATIONFROM', 'STATIONTO', 'NormalizedWagonType', 'INVOICENUMBER',
       'INVOICEDATE', 'NumOfWagons', 'ACTUALWEIGHT_sum', 'ACTUALWEIGHT_count',
       'COMMERCIALPLACEMENTTIME', 'REMOVALTIME', 'ATTRIBUTE4', 'FNRNUMBER',
       'NOOFUNITS', 'QUANTITY', 'SUB COMMODITY DETAIL NAME',
       'SUB COMMODITY DETAIL CODE', 'CommodityName',
       'SUB COMMODITY DETAIL NAME modified', 'CommodityName modified',
       'Unit Name', 'Consigner / Consignee Code', 'Siding/Station Code',
       'Type of Wagon', 'Commodity', 'Unloading / Loading', 'Free Hours',
       'TAT', 'Demurrage_Time', 'Demurrage_Cost', 'TAT_Categories',
       'CONSIGNOR', 'DEMURRAGECHARGEDUE', 'SIDING NAME', 'CONSIGNEE NAME',
       'CONSIGNOR NAME'],
      dtype='object')

In [131]:
merged_final_with_demurrage_siding_consignee.shape

(2835, 40)

In [132]:
def determine_status(row):
    from_station = row['STATIONFROM']
    to_station = row['STATIONTO']
    
    from_station_match = siding_code_df[(siding_code_df['STATION'] == from_station) & (siding_code_df['SIDING NAME'].str.contains('HINDALCO'))]
    to_station_match = siding_code_df[(siding_code_df['STATION'] == to_station) & (siding_code_df['SIDING NAME'].str.contains('HINDALCO'))]
    
    if not from_station_match.empty:
        return 'unloading'
    elif not to_station_match.empty:
        return 'loading'
    else:
        return ''

In [133]:
merged_final_with_demurrage_siding_consignee['STATUS'] = merged_final_with_demurrage_siding_consignee.apply(determine_status, axis=1)

In [134]:
merged_final_with_demurrage_siding_consignee.to_excel('merged_final_with_demurrage_siding_consignee.xlsx', index= False)

In [135]:
merged_final_with_demurrage_siding_consignee.shape

(2835, 41)

In [136]:
merged_final_with_demurrage_siding_consignee.columns

Index(['FNR', 'COMMODITYCODE', 'CONSIGNEENAME', 'DIVISION', 'STATION',
       'STATIONFROM', 'STATIONTO', 'NormalizedWagonType', 'INVOICENUMBER',
       'INVOICEDATE', 'NumOfWagons', 'ACTUALWEIGHT_sum', 'ACTUALWEIGHT_count',
       'COMMERCIALPLACEMENTTIME', 'REMOVALTIME', 'ATTRIBUTE4', 'FNRNUMBER',
       'NOOFUNITS', 'QUANTITY', 'SUB COMMODITY DETAIL NAME',
       'SUB COMMODITY DETAIL CODE', 'CommodityName',
       'SUB COMMODITY DETAIL NAME modified', 'CommodityName modified',
       'Unit Name', 'Consigner / Consignee Code', 'Siding/Station Code',
       'Type of Wagon', 'Commodity', 'Unloading / Loading', 'Free Hours',
       'TAT', 'Demurrage_Time', 'Demurrage_Cost', 'TAT_Categories',
       'CONSIGNOR', 'DEMURRAGECHARGEDUE', 'SIDING NAME', 'CONSIGNEE NAME',
       'CONSIGNOR NAME', 'STATUS'],
      dtype='object')

In [137]:
merged_final_with_demurrage_siding_consignee = merged_final_with_demurrage_siding_consignee[['FNR', 'DIVISION', 'STATION',
       'STATIONFROM', 'STATIONTO', 'NormalizedWagonType', 'INVOICENUMBER', 'COMMERCIALPLACEMENTTIME', 'REMOVALTIME',
       'INVOICEDATE', 'NumOfWagons', 'ACTUALWEIGHT_sum', 'ACTUALWEIGHT_count',
        'NOOFUNITS', 'QUANTITY', 'SUB COMMODITY DETAIL NAME modified', 'CommodityName modified',
       'Type of Wagon', 'Free Hours',
       'TAT', 'Demurrage_Time', 'Demurrage_Cost', 'TAT_Categories','CONSIGNOR',
       'DEMURRAGECHARGEDUE', 'SIDING NAME', 'CONSIGNEE NAME', 'CONSIGNOR NAME' ,'STATUS']]

In [138]:
merged_final_with_demurrage_siding_consignee['INVOICEDATE'] = pd.to_datetime(merged_final_with_demurrage_siding_consignee['INVOICEDATE'], format= '%d-%m-%Y')

In [62]:
# merged_final_with_demurrage_siding_consignee = merged_final_with_demurrage_siding_consignee[merged_final_with_demurrage_siding_consignee['FNR'] == 18032707097]

In [139]:
prevMonth_df = merged_final_with_demurrage_siding_consignee.copy()

In [140]:
prevMonth_df['INVOICEDATE'] = prevMonth_df['INVOICEDATE'] + pd.DateOffset(months=1)

In [141]:
prevMonth_df.rename(columns = {'NumOfWagons': 'PM_NumOfWagons', 'ACTUALWEIGHT_sum': 'PM_ACTUALWEIGHT_sum', 
                               'ACTUALWEIGHT_count':'PM_ACTUALWEIGHT_count', 'Free Hours': 'PM_Free Hours', 
                              'TAT': 'PM_TAT', 'Demurrage_Time': 'PM_Demurrage_Time', 'Demurrage_Cost': 'PM_Demurrage_Cost',
                              'DEMURRAGECHARGEDUE': 'PM_DEMURRAGECHARGEDUE'}, inplace= True)

In [142]:
concatenated_data_with_PM = merged_final_with_demurrage_siding_consignee.append([prevMonth_df])

C:\Users\nikhil.upadhyay-v\AppData\Local\Temp\ipykernel_22472\2227766286.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  concatenated_data_with_PM = merged_final_with_demurrage_siding_consignee.append([prevMonth_df])


In [143]:
concatenated_data_with_PM.to_excel('concatenated_data_with_PM.xlsx', index= False)

In [144]:
concatenated_data_with_PM.shape

(5670, 37)

In [145]:
concatenated_data_with_PM

,FNR,DIVISION,STATION,STATIONFROM,STATIONTO,NormalizedWagonType,INVOICENUMBER,COMMERCIALPLACEMENTTIME,REMOVALTIME,INVOICEDATE,NumOfWagons,ACTUALWEIGHT_sum,ACTUALWEIGHT_count,NOOFUNITS,QUANTITY,SUB COMMODITY DETAIL NAME modified,CommodityName modified,Type of Wagon,Free Hours,TAT,Demurrage_Time,Demurrage_Cost,TAT_Categories,CONSIGNOR,DEMURRAGECHARGEDUE,SIDING NAME,CONSIGNEE NAME,CONSIGNOR NAME,STATUS,PM_NumOfWagons,PM_ACTUALWEIGHT_sum,PM_ACTUALWEIGHT_count,PM_Free Hours,PM_TAT,PM_Demurrage_Time,PM_Demurrage_Cost,PM_DEMURRAGECHARGEDUE
0,18031612075,SBP,MAAL,NKCR,MAAL,BOXN,10,2020-05-04 17:30:00,2020-05-04 22:30:00,2020-05-03,59.0,4097.45,59.0,NaN,NaN,RUN OF MINES (ROM) COAL (F),Any commodity,BOXN,5.0,5.0,0.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM,NaN,loading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18032707097,SBP,MAAL,NKCR,MAAL,BOXN,14,2020-05-06 02:35:00,2020-05-06 10:35:00,2020-05-05,58.0,4053.00,58.0,NaN,NaN,RUN OF MINES (ROM) COAL (F),Any commodity,BOXN,5.0,8.0,3.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM,NaN,loading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18042409592,SBP,MAAL,NKCR,MAAL,BOXN,16,2020-05-07 04:20:00,2020-05-07 09:20:00,2020-05-06,59.0,3946.26,59.0,NaN,NaN,RUN OF MINES (ROM) COAL (F),Any commodity,BOXN,5.0,5.0,0.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM,NaN,loading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18061704498,SBP,MAAL,NKCR,MAAL,BOXN,21,2020-05-10 00:00:00,2020-05-11 05:00:00,2020-05-09,58.0,3972.20,58.0,NaN,NaN,RUN OF MINES (ROM) COAL (F),Any commodity,BOXN,5.0,29.0,24.0,NaN,21-40 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM,NaN,loading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18062105597,SBP,MAAL,NKCR,MAAL,BOXN,38,2020-03-17 12:20:00,2020-03-17 18:20:00,2020-03-16,59.0,4197.05,59.0,NaN,NaN,RUN OF MINES (ROM) COAL (F),Any commodity,BOXN,5.0,6.0,1.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM,NaN,loading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830,23082804032,CKP,OCIG,MAAL,OCIG,BOXN,2,2023-08-29 12:30:00,2023-08-29 20:30:00,2023-09-28,NaN,NaN,NaN,59.0,4070.0,FLY ASH,Any commodity,NaN,NaN,NaN,NaN,NaN,0-20 hrs,HILD,NaN,"ORISSA CEMENT LTD, RAJGANGPUR (JSG-ROU)","THIRD PARTY - ORISSA CEMENT LTD, RAJGANGPUR",NaN,unloading,51.0,3396.98,51.0,NaN,8.0,NaN,NaN,22950.0
2831,23082813580,JBP,HIMB,RSCR,HIMB,BOXN,26,2023-08-31 08:45:00,2023-08-31 13:45:00,2023-09-29,NaN,NaN,NaN,59.0,4000.0,RUN OFF MINES COAL(ROM),Any commodity,BOXN,NaN,NaN,NaN,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - MAHAN,MAHAN ALUMINIUM,NaN,loading,58.0,3774.24,58.0,5.0,5.0,0.0,0.0,NaN
2832,23082818992,DHN,HACG,BLCB,HACG,BOXN,22,2023-08-30 11:35:00,2023-08-30 21:30:00,2023-09-29,NaN,NaN,NaN,61.0,3894.0,RUN OFF MINES COAL(ROM),COAL,BOXN,NaN,NaN,NaN,NaN,0-20 hrs,CCL,NaN,HINDALCO LTD -RENUKOOT,RENUKOOT,NaN,loading,59.0,4090.34,59.0,7.0,10.0,3.0,28822.5,26550.0
2833,23082904040,DHN,HACG,MMRC,HACG,BOXN,30,2023-08-30 22:25:00,2023-08-31 03:55:00,2023-09-30,NaN,NaN,NaN,59.0,4012.0,BAUXITE,BAUXITE,BOXN,NaN,NaN,NaN,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD -RENUKOOT,RENUKOOT,NaN,loading,56.0,3735.95,56.0,7.0,6.0,-1.0,0.0,NaN


Previous Year calculation

In [146]:
prevYear_df = merged_final_with_demurrage_siding_consignee.copy()
prevYear_df['INVOICEDATE'] = prevYear_df['INVOICEDATE'] + pd.DateOffset(years=1)

In [147]:
prevYear_df.rename(columns = {'NumOfWagons': 'PY_NumOfWagons', 'ACTUALWEIGHT_sum': 'PY_ACTUALWEIGHT_sum', 
                               'ACTUALWEIGHT_count':'PY_ACTUALWEIGHT_count', 'Free Hours': 'PY_Free Hours', 
                              'TAT': 'PY_TAT', 'Demurrage_Time': 'PY_Demurrage_Time', 'Demurrage_Cost': 'PY_Demurrage_Cost',
                              'DEMURRAGECHARGEDUE': 'PY_DEMURRAGECHARGEDUE'}, inplace= True)

In [148]:
concatenated_data_with_PY = concatenated_data_with_PM.append([prevYear_df])

C:\Users\nikhil.upadhyay-v\AppData\Local\Temp\ipykernel_22472\2107594585.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  concatenated_data_with_PY = concatenated_data_with_PM.append([prevYear_df])


In [149]:
concatenated_data_with_PY.to_excel('concatenated_data_with_PY.xlsx', index= False)

In [150]:
concatenated_data_with_PY.head()

,FNR,DIVISION,STATION,STATIONFROM,STATIONTO,NormalizedWagonType,INVOICENUMBER,COMMERCIALPLACEMENTTIME,REMOVALTIME,INVOICEDATE,NumOfWagons,ACTUALWEIGHT_sum,ACTUALWEIGHT_count,NOOFUNITS,QUANTITY,SUB COMMODITY DETAIL NAME modified,CommodityName modified,Type of Wagon,Free Hours,TAT,Demurrage_Time,Demurrage_Cost,TAT_Categories,CONSIGNOR,DEMURRAGECHARGEDUE,SIDING NAME,CONSIGNEE NAME,CONSIGNOR NAME,STATUS,PM_NumOfWagons,PM_ACTUALWEIGHT_sum,PM_ACTUALWEIGHT_count,PM_Free Hours,PM_TAT,PM_Demurrage_Time,PM_Demurrage_Cost,PM_DEMURRAGECHARGEDUE,PY_NumOfWagons,PY_ACTUALWEIGHT_sum,PY_ACTUALWEIGHT_count,PY_Free Hours,PY_TAT,PY_Demurrage_Time,PY_Demurrage_Cost,PY_DEMURRAGECHARGEDUE
0,18031612075,SBP,MAAL,NKCR,MAAL,BOXN,10,2020-05-04 17:30:00,2020-05-04 22:30:00,2020-05-03,59.0,4097.45,59.0,NaN,NaN,RUN OF MINES (ROM) COAL (F),Any commodity,BOXN,5.0,5.0,0.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM,NaN,loading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18032707097,SBP,MAAL,NKCR,MAAL,BOXN,14,2020-05-06 02:35:00,2020-05-06 10:35:00,2020-05-05,58.0,4053.00,58.0,NaN,NaN,RUN OF MINES (ROM) COAL (F),Any commodity,BOXN,5.0,8.0,3.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM,NaN,loading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18042409592,SBP,MAAL,NKCR,MAAL,BOXN,16,2020-05-07 04:20:00,2020-05-07 09:20:00,2020-05-06,59.0,3946.26,59.0,NaN,NaN,RUN OF MINES (ROM) COAL (F),Any commodity,BOXN,5.0,5.0,0.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM,NaN,loading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18061704498,SBP,MAAL,NKCR,MAAL,BOXN,21,2020-05-10 00:00:00,2020-05-11 05:00:00,2020-05-09,58.0,3972.20,58.0,NaN,NaN,RUN OF MINES (ROM) COAL (F),Any commodity,BOXN,5.0,29.0,24.0,NaN,21-40 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM,NaN,loading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18062105597,SBP,MAAL,NKCR,MAAL,BOXN,38,2020-03-17 12:20:00,2020-03-17 18:20:00,2020-03-16,59.0,4197.05,59.0,NaN,NaN,RUN OF MINES (ROM) COAL (F),Any commodity,BOXN,5.0,6.0,1.0,NaN,0-20 hrs,NaN,NaN,HINDALCO LTD - ADITYA,ADITYA ALUMINIUM,NaN,loading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
master_data_ads = concatenated_data_with_PY[concatenated_data_with_PY['INVOICEDATE'] <= '2023-08-31']

In [152]:
master_data_ads.to_excel('master_data_ads.xlsx', index= False)